In [14]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go
from plotly.tools import FigureFactory as FF
import re
import os
from datetime import datetime, timedelta

df = pd.DataFrame()
os.getcwd()


'c:\\Users\\dscshap3808\\Documents\\my_scripts_new\\play4_activity'

In [11]:
# df = pd.read_clipboard(header = None)
# df[0] = pd.to_datetime(df[0].astype(str), yearfirst=True)
# px.line(df, x= 0, y = 1)

In [12]:
df = pd.read_csv('./siemens.csv')
re1 = re.compile(r'(?<=\.).+')
df.columns = [re1.findall(i)[0] for i in list(df.columns.to_numpy())]
df = df.dropna(subset=['activity_start_time'])


FileNotFoundError: [Errno 2] No such file or directory: './siemens.csv'

In [20]:
df['activity_end_time'] = pd.to_datetime(df['activity_end_time'])
df['activity_start_time'] = pd.to_datetime(df['activity_start_time'].fillna(0))
df['create_time'] = pd.to_datetime(df['create_time'])

In [21]:
"""
d单个activity的耗费时间.
"""
df['time_gap_inner'] = df['activity_end_time']-df['activity_start_time']
df['time_gap_inner'] = df['time_gap_inner'].fillna(
    timedelta(0)
    ).apply(timedelta.total_seconds)
df['time_gap_inner'] = [0 if i < 0 else i for i in df['time_gap_inner']]

"""
前一个activity到后一个activity相差的时间\. (单个时间内会有大量activity发生.)
"""
df = df.sort_values('activity_start_time')
df['time_gap_outer'] = df\
    .groupby(['user_id','inc_day'], dropna = False)['activity_start_time'].transform('diff')
df['time_gap_outer'] = df['time_gap_outer'].fillna(timedelta(0)).apply(timedelta.total_seconds)
df['time_gap_outer'] = [0 if i< 0 else i for i in df['time_gap_outer']]

"""
time gap of the day. 总计活跃时间, ( 未减除动作间的时间间隔 )
"""
mid1 = df.groupby(['inc_day', 'user_id']).agg(
    a = ('activity_start_time','min'), 
    b  = ('activity_start_time', 'max'))\
        [['a','b']].diff(axis = 1)['b'].apply(timedelta.total_seconds).reset_index()
mid1.columns = ['inc_day','user_id','time_gap_today']
df = df.merge(mid1, on = ['inc_day', 'user_id'], how = 'left')


In [22]:
def get_hour():
    list = ['activity_start_time', 'activity_end_time','create_time']
    
    for i in list: 
        listnew = i + '_hour'
        df[listnew] = df[i].dt.round('min').astype(str).str.slice(11,)
    return df
df = get_hour() 

In [23]:
df['activity_start_time_sec'] = df['activity_start_time'].astype(str).str.slice(11,)
df['activity_end_time_sec'] = df['activity_end_time'].astype(str).str.slice(11,)

In [24]:
# pd.set_option("display.max_rows", None, "display.max_columns", 55)
# df[df['user_id'] == 'BJMARS003'].query('time_gap_outer != 0')[[
#     'activity_start_time', 'activity_end_time','time_gap_inner'	,'time_gap_outer','time_gap_today']].\
#         sort_values('activity_start_time', ascending=False)

In [25]:
df[['ou_code',   'wms_warehouse_id', 'user_id', 'activity_type' ,
       'activity_start_time', 'activity_end_time',   
       'activity_id',  'lpn','sku_code', 'qty',
       'time_gap_inner','time_gap_outer','time_gap_today',
       'activity_start_time_hour', 'activity_end_time_hour', 'create_time_hour']]\
              .query("user_id == 'DGMHE' & activity_start_time == '2021-09-26 00:00:35' ")

,ou_code,wms_warehouse_id,user_id,activity_type,activity_start_time,activity_end_time,activity_id,lpn,sku_code,qty,time_gap_inner,time_gap_outer,time_gap_today,activity_start_time_hour,activity_end_time_hour,create_time_hour


In [26]:
pd.set_option("display.max_rows", 15, "display.max_columns", 15)

df[df['user_id'] == 'DGMHE'].query("time_gap_outer > 10.0") 

def check(id):
    print("===", df[df['user_id'] == id]['ou_name'].iloc[0],"===" , '\n', 'usr_id = ',id)
    for i, j  in enumerate([1,10, 30, 60, 600, 14400]):
        q = "time_gap_outer > " + str(j) 
        print(
            "和上个动作时间差距为" , j, '到' , [1,10, 30, 60, 600, 14400, '14400+'][i+1] , 
            '秒的动作数为: ', df[df['user_id'] == id].query(q).shape[0]) 

In [28]:
# df[['ou_code', 'ou_name', 'wms_warehouse_id', 'user_id', 'activity_type' ,
#        'activity_start_time', 'activity_end_time',   
#        'activity_id',  'lpn','sku_code', 
#        'time_gap_inner','time_gap_outer','time_gap_today',
#        'activity_start_time_hour', 'activity_end_time_hour', 'create_time_hour']]\
#               .query("user_id == 'DGMHE' & time_gap_outer>=   300.0")

In [ ]:
check('SIERF22')

In [ ]:
check('DSCJAXP0043')

In [ ]:
df[['ou_name', 'user_id']].drop_duplicates()

In [29]:
df[['ou_code',   
       
       'wms_warehouse_id', 'user_id', 'activity_type', 'activity_sub_type',
       'activity_start_time', 'activity_end_time', 'lpn', 'sku_code',
       'order_id', 'from_location', 'to_location', 'qty', 'create_time',
       'activity_id', 'inc_day',
       'time_gap_inner','time_gap_outer','time_gap_today',
       'activity_start_time_hour', 'activity_end_time_hour', 'create_time_hour','activity_start_time_sec']].\
              to_csv('./activi_for.csv', index=None, encoding='utf_8_sig')

In [ ]:
# df.groupby(['inc_day', 'user_id']).agg(
#     a = ('activity_start_time','min'), 
#     b  = ('activity_start_time', 'max'))[['a','b']].diff(axis = 1)['b']\
# .apply(timedelta.total_seconds).reset_index()
# df[df['user_id'] == 'DSCSZXP0325'][[
# 'activity_start_time','time_gap_inner','time_gap_outer','time_gap_today'
# ]].sort_values('activity_start_time')

In [ ]:
plt_data = df[['ou_code', 'ou_name', 'bg_code', 'bg_name_cn', 'customer_id',
       'customer_name', 'hour',
       'wms_warehouse_id', 'user_id', 'activity_type', 'activity_sub_type',
       'activity_start_time', 'activity_end_time', 'lpn', 'sku_code',
       'order_id', 'from_location', 'to_location', 'qty', 'create_time',
       'activity_id', 'inc_day',
       'time_gap_inner','time_gap_outer','time_gap_today',
       'activity_start_time_hour', 'activity_end_time_hour', 'create_time_hour']]

In [ ]:
import seaborn as sns

In [ ]:
sns.scatterplot('activity_start_time', 'time_gap_inner', \
    data= plt_data[plt_data['user_id'] == 'SIERF54'] )